In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from collections import Counter

In [ ]:
def find_distance(a, b, p=1):
    dim = len(a)
    distance = 0
    for d in range(dim):
        distance += abs(a[d] - b[d]) ** p
    distance = distance**(1/p)
    return distance

In [ ]:
def test_distance(test_point, train_points, y_train, k, p=1):
    distances = []
    for i in range(len(train_points)):
        distances.append(find_distance(test_point, train_points[i], p))
    df_distance = pd.DataFrame(data=distances, index=y_train.index,columns=["dist"])
    df_nn = df_distance.sort_values(by=["dist"], axis=0)[:k]
    return df_nn

In [ ]:
def return_most_common(y, df_nn):
    counter = Counter(y[df_nn.index])
    return counter.most_common()[0][0]

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
def knn_predict(X_train, y_train, X_test, k=5, p=1):
    y_predicted_data = []
    for test_point in X_test:
        distances = test_distance(test_point, X_train, y_train, k, p)
        prediction = return_most_common(y_train, distances)
        y_predicted_data.append(prediction)
    return y_predicted_data

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
glass_data = pd.read_csv("glass.csv")
glass_data.head()

In [ ]:
glass_data.isnull().sum()

In [ ]:
X = glass_data.drop("Type", axis=1)
X

In [ ]:
y = glass_data["Type"]
y

In [ ]:
def predict(X, y):
    np.random.seed(42)
    test_sizes = [0.3, 0.1]
    k_values = [3, 5, 7]
    p_values = [1, 2]
    print("="*40)
    for k in k_values:
        print(f"For k value = {k}")
        for size in test_sizes:
            print(f"For test size = {size}")
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=size, random_state=125)
            scaler = StandardScaler()
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)
            for p in p_values:
                if p == 2:
                    print("For Euclidean distance:")
                else:
                    print("For Manhattan distance:")
                y_preds = knn_predict(X_train, y_train, X_test, k=k, p=p)
                print("Accuracy score is: ", accuracy_score(y_test, y_preds))
            print("-"*30)
        print("="*40)

In [ ]:
def predict_leave_one_out(X, y):
    np.random.seed(42)
    k_values = [3, 5, 7]
    p_values = [1, 2]
    for k in k_values:
        print(f"For k value = {k}")
        print("For leave one out")
        for p in p_values:
            if p == 2:
                print("For Euclidean distance:")
            else:
                print("For Manhattan distance:")
            scores = []
            for i in range(100):
                random_idx = np.random.choice(range(len(X)))
                X_train, X_test = X.drop(random_idx, axis=0), X.iloc[random_idx]
                y_train, y_test = y.drop(random_idx, axis=0), y.iloc[random_idx]
                scaler = StandardScaler()
                X_train = scaler.fit_transform(X_train)
                X_test = X_test.to_numpy()
                X_test = X_test.reshape(1, -1)
                X_test = scaler.transform(X_test)
                y_preds = knn_predict(X_train, y_train, X_test, k=k, p=p)
                if y_preds[0] == y_test:
                    scores.append(1)
                else:
                    scores.append(0)
            scores = pd.DataFrame(data=scores, columns=["score"])
            scores = scores.value_counts()
            print("Percentage of correct predictions: ", (scores[0]/sum(scores))*100, "%")
        print("="*40)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
glass_data = pd.read_csv("glass.csv")
glass_data

In [ ]:
X = glass_data.drop("Type", axis=1)
X

In [ ]:
y = glass_data["Type"]
y

In [ ]:
predict(X, y)

In [ ]:
predict_leave_one_out(X, y)

In [ ]:
fruit_data = pd.read_table('fruits.txt')
fruit_data

In [ ]:
fruit_data.drop("fruit_subtype", axis=1, inplace=True)
fruits = fruit_data.drop("fruit_name", axis=1)

In [ ]:
fruits

In [ ]:
X = fruits.drop("fruit_label", axis=1)
X

In [ ]:
y = fruits["fruit_label"]
y

In [ ]:
predict(X, y)

In [ ]:
predict_leave_one_out(X, y)